# Rosetta Filter API

@Author: 黄健 @E-mail: jian.huang@xtalpi.com

@Proofread: 吴炜坤 @email：weikun.wu@xtalpi.com

更多参考: https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Filters/Filters-RosettaScripts



本章节将详细介绍Pyrosetta中一些常用的filter的使用，并给出示例。请读者根据自己需求，需要使用时进行查询即可。

In [1]:
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
init()

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.26+release.b308454c455dd04f6824cc8b23e54bbb9be2cdd7 2021-07-02T13:01:54] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r288 2021.26+release.b308454c455 b308454c455dd04f6824cc8b23e54bbb9be2cdd7 http://www.pyrosetta.org 2021-07-02T13:01:54
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=85585984 seed_offset=0 real_seed=85585984 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=85585984 RG_type=mt19937


### 1. SimpleMetricFilter（简单介绍）
基于SimpleMetric计算的值判断是否保留构象的过滤器。

In [20]:
from pyrosetta.rosetta.protocols.simple_filters import SimpleMetricFilter
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
from pyrosetta.rosetta.core.simple_metrics.metrics import SasaMetric
from pyrosetta.rosetta.protocols.simple_filters import comparison_type

# 读取pose
pose = pose_from_pdb('./data/1ubq_clean.pdb')
print(pose.pdb_info())

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB
PDB file name: ./data/1ubq_clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0076    A 0001  -- 0076  |   0076 residues;    01234 atoms
                           TOTAL |   0076 residues;    01234 atoms



In [21]:
# 定义SimpleMetrics计算器
sasa_sel = ResidueIndexSelector('1-76')  # 比如计算1-76号残基每个残基的sasa值
sasa_metrics = SasaMetric(sasa_sel)

In [23]:
# 定义SimpleMetricFilter
sasa_filter = SimpleMetricFilter()
sasa_filter.set_simple_metric(sasa_metrics)  # 设定SimpleMetrics
sasa_filter.set_cutoff(500)  # 设定截断半径;
sasa_filter.set_comparison_type(comparison_type.gt) # gt 等于great than, filter的判断逻辑
sasa_filter.apply(pose)

protocols.simple_filters.SimpleMetricFilter: {0} 4738.4 gt 500 ?
protocols.simple_filters.SimpleMetricFilter: {0} Filter passed: 1


True

**点评**:其实在python操作中，完全没必要去设定SimpleMetricFilter。直接根据SimpleMetric返回的内容进行判断True or False。这种python语言中是非常容易实现的。
此处仅做一个简单的案例，阐明SimpleMetricFilter的基本作用。

### 2. Basic Filters
此部分根据官方的Filter文档介绍，ResidueCount和NetCharge的用法。

#### 2.1 ResidueCount
根据残基类型、残基性质、Pack状态的计数/计频filter，可设置过滤阈值。当多个性质或类型被设置是，处理的逻辑是“或”。

In [29]:
from pyrosetta.rosetta.protocols.simple_filters import ResidueCountFilter

# 添加过滤的残基性质类型
res_count_filter = ResidueCountFilter()
res_count_filter.add_residue_property_by_name('POLAR')
res_count_filter.score(pose)

41.0

In [32]:
from pyrosetta.rosetta.core.chemical import ResidueTypeSet
from pyrosetta.rosetta.core.chemical import ChemicalManager
from pyrosetta.rosetta.core.conformation import ResidueFactory

# 获取ResidueTypeSet
chm = ChemicalManager.get_instance()
residue_type_sets = chm.residue_type_set("fa_standard")

# 添加过滤的残基名(ALA)。
res_count_filter = ResidueCountFilter()
res_count_filter.add_residue_type_by_name(residue_type_sets, 'ALA')
res_count_filter.score(pose)

2.0

#### 2.2 NetCharge
基于蛋白序列总电荷值的过滤器，NetCharge设定LYS和ARG残基电荷值为+1，酸性残基ASP和GLU电荷值为-1。

In [35]:
from pyrosetta.rosetta.protocols.simple_filters import NetChargeFilter
netcharge = NetChargeFilter()
netcharge.apply(pose)

protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 6
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 11
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  GLU 16
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  GLU 18
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  ASP 21
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  GLU 24
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 27
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 29
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  ASP 32
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 33
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  GLU 34
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  ASP 39
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  ARG 42
protocols.simple_filters.NetChargeFilter: {0} AA:  +1  LYS 48
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  GLU 51
protocols.simple_filters.NetChargeFilter: {0} AA:  -1  ASP 52
protocols

True

### 3. Energy/Score Filters

#### 3.1 ScoreTypeFilter

#### 3.2 TaskAwareScoreType

#### 3.3 BindingStrain

#### 3.4 ConstraintScore

#### 3.5 EnergyPerResidue

#### 3.6 Residue Interaction Energy

#### 3.7 ResidueSetChainEnergy

#### 3.8 ScorePoseSegmentFromResidueSelectorFilter

### 4. Distance Filter

#### 4.1 ResidueDistance

#### 4.2 AtomicContact

#### 4.3 AtomicContactCount

### 5. Sequence analysis

#### 5.1 LongestContinuousPolarSegment

#### 5.2 LongestContinuousApolarSegment

#### 5.3 SequenceDistanceFilter

### 6. Alignment analysis

#### 6.1 AlignmentAAFinder

#### 6.2 AlignmentGapInserter

### 7. Geometry